<a href="https://colab.research.google.com/github/specM7/DSGP_Group_33_Brain_Tumor_Predictor/blob/main/Meningioma_Adrian_2425482.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 AI-Powered Brain Tumor Predictor for Meningioma(Adrian)


In [64]:
import os, glob, cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

print("TensorFlow:", tf.__version__)


TensorFlow: 2.19.0


In [73]:
from google.colab import files

uploaded = files.upload()        # PICK the real DataSet.zip from your PC

zip_name = list(uploaded.keys())[0]
print("Uploaded:", zip_name)

# Unzip into /content/DataSet_raw
!rm -rf /content/DataSet_raw
!mkdir -p /content/DataSet_raw
!unzip -q "$zip_name" -d /content/DataSet_raw

print("\nTop level under /content/DataSet_raw:")
!ls /content/DataSet_raw

print("\nAny 'Training' folders:")
!find /content/DataSet_raw -maxdepth 5 -type d -iname "Training"


Saving DataSet.zip to DataSet (5).zip
Uploaded: DataSet (5).zip

Top level under /content/DataSet_raw:
DataSet

Any 'Training' folders:
/content/DataSet_raw/DataSet/Training


In [74]:
# PASTE the exact Training path from the previous cell:
training_path = "/content/DataSet_raw/DataSet/Training"   # example, edit to your real one

raw_root = os.path.dirname(training_path)
print("training_path:", training_path)
print("raw_root:", raw_root)
print("RAW training folders:", os.listdir(training_path))

img_size = (224, 224)
batch_size = 32

train_dir = training_path
test_dir  = os.path.join(raw_root, "Testing")
print("train_dir:", train_dir)
print("test_dir:", test_dir)

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=42,
    color_mode="grayscale"
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=42,
    color_mode="grayscale"
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    color_mode="grayscale"
)

print("Classes:", train_ds.class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.shuffle(1000).prefetch(AUTOTUNE)
val_ds   = val_ds.prefetch(AUTOTUNE)
test_ds  = test_ds.prefetch(AUTOTUNE)


training_path: /content/DataSet_raw/DataSet/Training
raw_root: /content/DataSet_raw/DataSet
RAW training folders: ['glioma', 'notumor', 'pituitary', 'meningioma']
train_dir: /content/DataSet_raw/DataSet/Training
test_dir: /content/DataSet_raw/DataSet/Testing
Found 5712 files belonging to 4 classes.
Using 4570 files for training.
Found 5712 files belonging to 4 classes.
Using 1142 files for validation.
Found 1826 files belonging to 4 classes.
Classes: ['glioma', 'meningioma', 'notumor', 'pituitary']


In [75]:
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=img_size + (1,)),
    layers.Conv2D(32, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(4, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25
)

test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_4 (Rescaling)         │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 222, 222, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,168,900 (42.61 MB)

 Trainable params: 11,168,900 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 519s 4s/step - accuracy: 0.5270 - loss: 1.0792 - val_accuracy: 0.8039 - val_loss: 0.5544
Epoch 2/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 522s 4s/step - accuracy: 0.7680 - loss: 0.5961 - val_accuracy: 0.8371 - val_loss: 0.4404
Epoch 3/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 525s 4s/step - accuracy: 0.8233 - loss: 0.4674 - val_accuracy: 0.8371 - val_loss: 0.4045
Epoch 4/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 519s 4s/step - accuracy: 0.8554 - loss: 0.3914 - val_accuracy: 0.8800 - val_loss: 0.3449
Epoch 5/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 523s 4s/step - accuracy: 0.8697 - loss: 0.3433 - val_accuracy: 0.8940 - val_loss: 0.3288
Epoch 6/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 521s 4s/step - accuracy: 0.8980 - loss: 0.2935 - val_accuracy: 0.9028 - val_loss: 0.3069
Epoch 7/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 517s 4s/step - accuracy: 0.9091 - loss: 0.2510 - val_accuracy: 0.9019 - val_loss: 0.2778
Epoch 8/25
143/143 ━━━━━━━━━━━━━━━━━━━━ 521s 4s/step - accuracy: 0.9113 - loss: 0.2283 - val_accu

In [1]:
model.save("brain_tumor_mri_model.h5")


NameError: name 'model' is not defined